<a href="https://colab.research.google.com/github/rahilsaiyed1711/AboutUsPage/blob/main/Adani_project_11_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install flask_ngrok

ERROR: Operation cancelled by user


In [ ]:
import pandas as pd
from flask import Flask, request, jsonify
from google.colab.output import eval_js
from IPython.display import display, Javascript

# Create sample CSV files for different blocks
blocks_data = {
    "Block-1": {
        'Block': ["Block-1"] * 21,
        'Inverter make': ["Sungrow-Central Inverter"] * 21,
        'Model': ["SG3125HV"] * 21,
        'Inverter No.': [1] * 21,
        'SCB': list(range(1, 22)),
        'No. of string': [12, 12, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11],
        'Modules/strings': [60] * 21,
        'Total Modules': [720, 720, 660, 660, 660, 660, 660, 660, 660, 660, 660, 660, 660, 660, 660, 660, 660, 660, 660, 660, 660],
        'Module make': ["Jinergy"] * 21,
        'Model': ["JNMP72-325", "JNMP72-325", "JNMP72-325", "JNMP72-325", "JNMP72-325", "JNMP72-325", "JNMP72-325", "JNMP72-325", "JNMP72-330", "JNMP72-325", "JNMP72-325", "JNMP72-325", "JNMP72-325", "JNMP72-325", "JNMP72-325", "JNMP72-325", "JNMP72-325", "JNMP72-330", "JNMP72-325", "JNMP72-325", "JNMP72-325"],
        'P (Wp)': [325] * 21,
        'DC Capacity(kW)': [234, 234, 214.5, 214.5, 214.5, 214.5, 214.5, 214.5, 214.5, 214.5, 214.5, 214.5, 214.5, 214.5, 214.5, 214.5, 214.5, 214.5, 214.5, 214.5, 214.5],
        'Y connector if connected': ["No"] * 21,
        'Orientation(Fixed/Seasonal/Tracker)': ["Fixed"] * 21,
    },
    # Add more blocks here
}

for block, data in blocks_data.items():
    df = pd.DataFrame(data)
    df.to_csv(f'{block.lower()}.csv', index=False)

app = Flask(__name__)

@app.route('/')
def home():
    return """
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>PV Form</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f0f0f0;
            margin: 0;
            padding: 20px;
        }

        h1 {
            color: #333;
            text-align: center;
        }

        button {
            display: block;
            width: 150px;
            padding: 10px;
            margin: 20px auto;
            background-color: #007BFF;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            transition: background-color 0.3s ease;
        }

        button:hover {
            background-color: #0056b3;
        }

        #tabs {
            display: flex;
            justify-content: center;
            margin-top: 20px;
        }

        .tab {
            padding: 10px 20px;
            margin: 0 5px;
            background-color: #007BFF;
            color: white;
            border-radius: 5px;
            cursor: pointer;
            transition: background-color 0.3s ease;
        }

        .tab:hover {
            background-color: #0056b3;
        }

        #data {
            margin-top: 20px;
        }

        table {
            width: 100%;
            border-collapse: collapse;
            margin-top: 20px;
            background-color: white;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }

        th, td {
            padding: 10px;
            text-align: left;
            border: 1px solid #ddd;
        }

        th {
            background-color: #007BFF;
            color: white;
        }

        tr:nth-child(even) {
            background-color: #f2f2f2;
        }

        td[contenteditable="true"] {
            background-color: #fffdd0;
            cursor: pointer;
        }

        td[contenteditable="true"]:hover {
            background-color: #fffacd;
        }
    </style>
</head>
<body>
    <h1>PV Form</h1>
    <button onclick="loadData()">Load Data</button>
    <div id="tabs">
        <div class="tab" onclick="loadBlock('block-1')">Block 1</div>
        <div class="tab" onclick="loadBlock('block-2')">Block 2</div>
        <div class="tab" onclick="loadBlock('block-3')">Block 3</div>
        <div class="tab" onclick="loadBlock('block-4')">Block 4</div>
    </div>
    <div id="data"></div>
    <button onclick="saveData()">Save Data</button>

    <script>
    let currentBlock = 'block-1';

    function loadBlock(block) {
        currentBlock = block;
        loadData();
    }

    function loadData() {
        fetch(`/load_data?block=${currentBlock}`)
            .then(response => response.json())
            .then(data => {
                let table = '<table><tr>';
                for (let key in data[0]) {
                    table += `<th>${key}</th>`;
                }
                table += '</tr>';
                data.forEach(row => {
                    table += '<tr>';
                    for (let key in row) {
                        table += `<td contenteditable="true">${row[key]}</td>`;
                    }
                    table += '</tr>';
                });
                table += '</table>';
                document.getElementById('data').innerHTML = table;
            });
    }

    function saveData() {
        let data = [];
        let table = document.querySelector('table');
        let headers = Array.from(table.querySelectorAll('th')).map(th => th.textContent);
        let rows = table.querySelectorAll('tr');
        for (let i = 1; i < rows.length; i++) {
            let row = {};
            let cells = rows[i].querySelectorAll('td');
            headers.forEach((header, index) => {
                row[header] = cells[index].textContent;
            });
            data.push(row);
        }
        fetch(`/save_data?block=${currentBlock}`, {
            method: 'POST',
            headers: {
                'Content-Type': 'application/json',
            },
            body: JSON.stringify(data),
        })
        .then(response => response.json())
        .then(result => alert(result.message));
    }
    </script>
</body>
</html>
    """

@app.route('/load_data')
def load_data():
    block = request.args.get('block', 'block-1')
    df = pd.read_csv(f'{block}.csv')
    return jsonify(df.to_dict(orient='records'))

@app.route('/save_data', methods=['POST'])
def save_data():
    block = request.args.get('block', 'block-1')
    data = request.json
    df = pd.DataFrame(data)
    df.to_csv(f'{block}.csv', index=False)
    return jsonify({"message": "Data saved successfully"})

# Function to create a public URL
def create_public_url():
    return eval_js("""
        (async () => {
            const url = await google.colab.kernel.proxyPort(5000);
            return url;
        })()
    """)

# Run the Flask app
if __name__ == '__main__':
    public_url = create_public_url()
    print(f"Public URL: {public_url}")
    app.run(port=5000)


Public URL: https://8tenunl6599-496ff2e9c6d22116-5000-colab.googleusercontent.com/
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Jul/2024 06:44:31] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2024 06:44:31] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2024 06:44:34] "GET /load_data?block=block-1 HTTP/1.1" 200 -
ERROR:__main__:Exception on /load_data [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py

In [14]:
files.download('solar_plants.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>